In [1]:
##############################################
# Task subdivision based on reachability     #
##############################################

# Packages
import numpy as np
import time
import rospy
import rospkg
from corrective_shared_autonomy.TaskModels.DMPLWRhardcoded import HybridSegment
from corrective_shared_autonomy.TaskModels.DMPLWRhardcoded import DMPLWRhardcoded
from core_robotics.PyBSpline import BSplineSurface
from scipy.spatial.transform import Rotation as ScipyR
from scipy.spatial.transform import Slerp

rospy.init_node('subdivisiontester', anonymous=True)

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
##############################################
# Helper Functions                           #
##############################################


# starting vals is a list of starting vals
# ending vals is a list of ending vals
# num_pts is how many interp values
# quat_vars is the index of any starts of quaternions so they can be SLERPed instead of LERPed
# super_pos_vars is the starting index for 2-pairs of variables which should have an added sine wave (e.g., rubbing)
# super_pos_freq is the frequency for the starting index pairs
# super_pos_amp is the amplitude for the starting index pairs
def interpMultD(starting_vals,ending_vals,num_pts,quat_vars=[], super_pos_vars = [], super_pos_freq=[], super_pos_amp = []):
    vals = []
    
    # LERP ALL VARIABLES
    for ii in range(0,num_pts):
       
        # super is to potentially add a circular motion on top of the LERP motion
        freq_super = 0.0
        amp_super = 0.0
        added_sine = np.zeros((np.shape(starting_vals)))
        for jj in range(0,len(super_pos_vars)):
            amp = super_pos_amp[jj]
            freq = super_pos_freq[jj]
            added_sine[super_pos_vars[jj]] = np.sin(np.pi*float(ii)/num_pts)*amp*np.sin(freq*float(ii)/2*np.pi)
            added_sine[super_pos_vars[jj]+1] = np.sin(np.pi*float(ii)/num_pts)*amp*np.cos(freq*float(ii)/2*np.pi)
        c_i = float(ii)/float(num_pts) # interpolation coefficient
        vals.append(list(c_i*(e-s)+s+a for s,e,a in zip(starting_vals,ending_vals,added_sine)))
    
    vals = np.asarray(vals) # currently,num_samples x num_vars
    
    # SLERP QUATERIONS
    for jj in range(0,len(quat_vars)):
        starting_q = np.array(starting_vals[quat_vars[jj]:quat_vars[jj]+4])
        ending_q = np.array(ending_vals[quat_vars[jj]:quat_vars[jj]+4])
        key_times = [0,num_pts-1]
        key_rots = ScipyR.from_quat(np.concatenate((starting_q.reshape((1,4)),ending_q.reshape((1,4))),axis=0))
        slerper = Slerp(key_times, key_rots)
        
        pts = list(range(0,num_pts))
        slerped = slerper(pts)
        vals[:,quat_vars[jj]:quat_vars[jj]+4] = np.asarray(slerped.as_quat())
    
    return vals.T # num vars x num_samples


In [3]:
##############################################
# Create data for testing                    #
##############################################
rospack = rospkg.RosPack()
uliconfigpath = rospack.get_path('uli_config')


t_surf = np.array([0.43070987363200586, -0.013269419259123479, 0.06385816493811966])
R_surf = np.array([0.008339958880240922, 0.004026424369577458, -0.029073916288389654, 0.9995343617826891])
R_scisurf = ScipyR.from_quat(R_surf)
surf_file = uliconfigpath + '/registration_models/IRC_piece1.csv'

ang_relative_to_surface = 90 # degrees
R_tool_surf = ScipyR.from_euler('zx',[ang_relative_to_surface,0],degrees=True)


surf = BSplineSurface()
surf.loadSurface(surf_file)


starting = [[0.1, 0.1], [0.3, 0.1]]
ending = [[0.1, 0.9], [0.3, 0.9]]
samps = [25,25]

trajectories = []

for ii in range(0,len(starting)):
    temp_vars = interpMultD(starting[ii],ending[ii],samps[ii])
    data_temp = np.zeros((7,samps[ii]))
    
    for jj in range(0,np.shape(temp_vars)[1]):
        # get surface coordinates
        r, n_hat, r_u_norm, r_v_norm = surf.calculate_surface_point(temp_vars[0,jj],temp_vars[1,jj])
        
        R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))

        # apply surface transform
        q_new = np.round((R_scisurf * R_surf * R_tool_surf).as_quat(),3)
        r_new = R_scisurf.apply(r) + t_surf
        
        data_temp[0:3,jj] = r_new
        data_temp[3:,jj] = q_new
    
    trajectories.append(data_temp)

    
# print(trajectories)

In [18]:
from corrective_shared_autonomy.srv import IK
from geometry_msgs.msg import Pose
from std_msgs.msg import String
from sensor_msgs.msg import JointState

def ang_btwn_quats(q1,q2):
    # assumes xyzw notation
    R_1 = ScipyR.from_quat(q1)
    R_2 = ScipyR.from_quat(q2)
    ang_diff = np.linalg.norm(((R_1.inv()) * R_2).as_rotvec())
    return ang_diff

def queryReachability(pos,quat,urdf,baselink,eelink, pos_tol, quat_tol, jointnames):
    rospy.wait_for_service('/corrective_shared_autonomy/solve_ik')
    try:
        ik_soln = rospy.ServiceProxy('/corrective_shared_autonomy/solve_ik', IK)

        urdf_file = String()
        urdf_file.data = urdf
        base = String()
        base.data = baselink
        ee = String()
        ee.data = eelink

        des_pose = Pose()
        des_pose.orientation.x = quat[0]
        des_pose.orientation.y = quat[1]
        des_pose.orientation.z = quat[2]
        des_pose.orientation.w = quat[3]
        des_pose.position.x = pos[0]
        des_pose.position.y = pos[1]
        des_pose.position.z = pos[2]

        # package jointnames
        jnames = []
        for ii in range(0,len(jointnames)):
            jnames.append(String(jointnames[ii]))


        startt = time.time()
#         print(des_pose)
        resp = ik_soln(urdf_file,base,ee,des_pose,jnames)
        # print("Time: ",time.time()-startt)
        # print(resp)

        # convert response to numpy
        soln_pos = resp.soln_pose.position
        soln_pos_np = np.array([soln_pos.x, soln_pos.y, soln_pos.z])
        soln_quat = resp.soln_pose.orientation
        soln_quat_np = np.array([soln_quat.x, soln_quat.y, soln_quat.z, soln_quat.w])

        pos_error = np.linalg.norm(soln_pos_np-pos)
        quat_error = ang_btwn_quats(soln_quat_np,quat)

        if (pos_error<pos_tol and quat_error<quat_tol):
            return True, np.array(resp.soln_joints.data)
        else:  
            return False, None           

    except rospy.ServiceException as e:
#         print("Service call failed: %s"%e)
        return False, None


##############################################
# Run Splitting algorithm                    #
##############################################

# input to algorithm is a list of 7xsamps trajectories and a urdf
rospack = rospkg.RosPack()
configpath = rospack.get_path('uli_config') + '/../Actuation/config/'
urdf_file = configpath + 'panda.urdf'
# urdf_file = '/home/mike/Documents/ultrasound_reachability/src/reachability_analysis/urdfs/panda_arm_us.urdf'

print(urdf_file)

# TODO: include the reachability stuff
# TODO: create a mask of not done, excluded, done

# Plot full trajectory
from core_robotics.dataviz import threedplot, highlightedthreedplot

x = []
y = []
z = []
for ii in range(0,len(trajectories)):
    x.extend(trajectories[ii][0,:])
    y.extend(trajectories[ii][1,:])
    z.extend(trajectories[ii][2,:])
x = np.array(x)
y = np.array(y)
z = np.array(z)
xs = []
ys = []
zs = []
highlightedthreedplot(x, y, z, xs, ys, zs, marker_size = 5.0)


###############################################
# Step 0: set up mask                         #
###############################################

# TODO: enum
# 0 is not done, 1 is proposed, 2 is completed

taskmask = []
for ii in range(0,len(trajectories)):
    trajmask = np.zeros((np.shape(trajectories[ii])[1],))
    taskmask.append(trajmask)

###############################################
# Step 1: cull based on reachability          #
###############################################

# set up reachability service call
jointnames = ['panda_joint1','panda_joint2', 'panda_joint3', 'panda_joint4', 'panda_joint5', 'panda_joint6', 'panda_joint7']
baselink = 'panda_link0'
eelink = 'panda_orbital'
pos_tol = 0.002
quat_tol = 0.03

# call for each element of each trajectory
for ii in range(0,len(trajectories)):
    for jj in range(0,np.shape(trajectories[ii])[1]):
        pos = trajectories[ii][0:3,jj].flatten()
        quat = trajectories[ii][3:7,jj].flatten()
        success, jangles = queryReachability(pos,quat,urdf_file,baselink,eelink, pos_tol, quat_tol, jointnames)
        if success:
            taskmask[ii][jj]=1

x_good = []
y_good = []
z_good = []
x_bad = []
y_bad = []
z_bad = []
for ii in range(0,len(trajectories)):
    for jj in range(0,np.shape(trajectories[ii][1])):
        if taskmask
        x.extend(trajectories[ii][0,:])
        y.extend(trajectories[ii][1,:])
        z.extend(trajectories[ii][2,:])
x = np.array(x)
y = np.array(y)
z = np.array(z)
xs = []
ys = []
zs = []
highlightedthreedplot(x, y, z, xs, ys, zs, marker_size = 5.0)


###############################################
# Step 2: cull based on discounted path length#
###############################################

###############################################
# Step 3: cull manipulability                 #
###############################################

/home/mike/Documents/demo/src/panda_uli_demo/ULIConfig/../Actuation/config/panda.urdf
[array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 0., 0., 0., 0., 0.]), array([0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       0., 1., 0., 0., 0., 0., 0., 0.])]
